In [8]:
install.packages("rdatacite")
install.packages("httr")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [9]:
library(jsonlite)
library(dplyr)
library(stringr)
library(httr)
library("memoise")



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘memoise’

The following object is masked from ‘package:httr’:

    timeout



In [10]:
call_api<-function(rows, cursor){
    response <- fromJSON(paste0('https://query.eventdata.crossref.org/events?rows=',rows,'&filter=source:datacite&cursor=',cursor))
    return(response)
}


In [11]:
get_data_pub_links<-function(rows) {
    start <- fromJSON(paste0('https://query.eventdata.crossref.org/events?rows=',rows,'&filter=source:datacite'))
    data <- start$message$events
    total<- start$message$`total-results`
    cursor <- start$message$`next-cursor`
    repeat {   
        print(cursor)
        response <- call_api(rows,cursor)
        cursor <- response$message$`next-cursor`
        print(cursor)
        if (nrow(data)+rows >= total) {
            print("Well done!")
            break
        } 
        else {
            data <-rbind(data,response$message$events )
            print(nrow(data))
            print("another line")
       }
    }
    return(data)
}


In [12]:
data<-get_data_pub_links(2200)


[1] "6b7added-d95b-4d75-b73c-2e9459f2193a"
[1] "81f6af8f-e149-43ff-a195-cd6a26757fd6"
[1] 4400
[1] "another line"
[1] "81f6af8f-e149-43ff-a195-cd6a26757fd6"
[1] "f73cdef7-18b2-443e-90a7-71110098e411"
[1] 6600
[1] "another line"
[1] "f73cdef7-18b2-443e-90a7-71110098e411"
[1] "0f5bdbef-d236-4c4d-b1f4-84e2c5ecdbac"
[1] 8800
[1] "another line"
[1] "0f5bdbef-d236-4c4d-b1f4-84e2c5ecdbac"
[1] "30f2f807-8643-4580-8088-ae90c69bf911"
[1] 11000
[1] "another line"
[1] "30f2f807-8643-4580-8088-ae90c69bf911"
[1] "5963fcbe-f71b-45e1-b60d-65972159a892"
[1] 13200
[1] "another line"
[1] "5963fcbe-f71b-45e1-b60d-65972159a892"
[1] "68303866-f393-4858-9b6b-53cccda0c52e"
[1] 15400
[1] "another line"
[1] "68303866-f393-4858-9b6b-53cccda0c52e"
[1] "944e9454-bf9b-4ee7-b91b-26bf99b63669"
[1] 17600
[1] "another line"
[1] "944e9454-bf9b-4ee7-b91b-26bf99b63669"
[1] "76015082-0343-4adc-9beb-74f3dfa33440"
[1] 19800
[1] "another line"
[1] "76015082-0343-4adc-9beb-74f3dfa33440"
[1] "00f8f92a-132e-45cc-a918-7ed1deaae26

ERROR: Error in rbind(deparse.level, ...): numbers of columns of arguments do not match


In [159]:
get_ra<-function(prefix){
    ra<-fromJSON(paste0("https://api.datacite.org/prefixes/",prefix))
    return((ra$data$attributes$`registration-agency`))
}

In [161]:
create_data_pack<- function(data){
    data <- data %>% 
        mutate(occurred_at = as.Date(occurred_at),
               timestamp = as.Date(timestamp),
               relation_type_id = as.factor(relation_type_id),
               message_action = as.factor(message_action),
               obj_prefix = paste0(str_extract(obj_id,'(10\\.\\d{4,5})'),"/"),
               subj_prefix = paste0(str_extract(subj_id,'(10\\.\\d{4,5})'),"/"),
               obj_doi = str_extract(obj_id,'(10\\.\\d{4,5}\\/.+)\\z'),
               subj_doi = str_extract(subj_id,'(10\\.\\d{4,5}\\/.+)\\z'),
#                obj_ra = as.character(get_ra(obj_prefix)),
#                obj_dc = get_dcdatacentre(obj_id, obj_ra),
               message_action = as.factor(message_action),
               source_id = as.factor(source_id))
    print(nrow(data))
    return(data)
}
data_pack<-create_data_pack(data)
head(data_pack)

[1] 22000


In [3]:
file<-paste0("../data/",Sys.Date(),"_links_dc_to_xref.Rda")
print(file)
save(data_pack,file=file)

[1] "../data/2018-03-15_links_dc_to_xref.Rda"


[1] "2018-03-15"

## Processing Adding Datcentre

https://api.datacite.org/works?query=prefix:10.5256&rows=1

In [4]:
load('../data/2018-03-15_links_dc_to_xref.Rda')

Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file '../data/2018-03-15_links_dc_to_xref.Rda', probable reason 'No such file or directory'”

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [182]:
get_dcdatacentrebyprefix<- function(obj_prefix){
    prefix <- obj_prefix
    url <- sprintf("https://api.datacite.org/prefixes/%s/", prefix)
    
    dc_info<-fromJSON(url)
#     return(head(dc_info$meta$`data-centers`,1)$title)
    id <- head(dc_info$data$relationships$clients$data,1)$id
    if(!is.null(id)){
        print(id)
        client<-fromJSON(paste0('https://api.datacite.org/clients/',id))
        clientinfo<-client$data$attributes$name
    }else{
        clientinfo<-"No client"
    }

    return(clientinfo)
}


In [183]:
get_xrdatacentrebyprefix<- function(subj_prefix){
    dc_info<-fromJSON(paste0('https://api.crossref.org/prefixes/',(subj_prefix)))
    return(head(dc_info$message$name,1))
}

In [184]:
get_datacentreName <- function(prefix){
    ra<- get_ra(prefix)
    print(ra)
    print(prefix)
    if(ra == "DataCite"){
        r<-get_dcdatacentrebyprefix(prefix)
        print(prefix)
    }
    else{
        r<-get_xrdatacentrebyprefix(prefix)
    }
    return(r)
}



In [185]:
get_listOfPrefix_Names<-function(data){
    prefixes<-as.data.frame(unique(data$obj_prefix))
    prefixes$value<-prefixes$`unique(data$obj_prefix)`
    colnames(prefixes)[2] <- "obj_prefix"


    prefixes <- prefixes %>% 
        rowwise() %>% 
        mutate(
            obj_dc = as.factor(get_datacentreName(obj_prefix)) 
        )
    return(prefixes)
}
# get_listOfPrefix_Names(data_pack)

In [191]:
dd<-get_listOfPrefix_Names(data_pack)

[1] "DataCite"
[1] 10.13140/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "rg.rg"
[1] 10.13140/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "DataCite"
[1] 10.5061/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "cdl.dryad"
[1] 10.5061/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/


Warning message in mutate_impl(.data, dots):
“Unequal factor levels: coercing to character”Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "DataCite"
[1] 10.7930/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "cdl.usgcrp"
[1] 10.7930/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "DataCite"
[1] 10.5195/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "cdl.pitt"
[1] 10.5195/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/


...

Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "DataCite"
[1] 10.13009/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "inist.onera"
[1] 10.13009/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "DataCite"
[1] 10.25423/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/
[1] "cmcc.opa"
[1] 10.25423/
379 Levels: 10.11577/ 10.11586/ 10.11588/ 10.1184/ 10.12682/ ... 10.7945/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

In [194]:
get_listOfPrefix_Names_sub<-function(data){
    prefixes<-as.data.frame(unique(data$subj_prefix))
    prefixes$value<-prefixes$`unique(data$subj_prefix)`
    colnames(prefixes)[2] <- "subj_prefix"


    prefixes <- prefixes %>% 
        rowwise() %>% 
        mutate(
            subj_dc = as.factor(get_datacentreName(subj_prefix)) 
        )
    return(prefixes)
}
# get_listOfPrefix_Names(data_pack)

In [195]:
dd2<-get_listOfPrefix_Names_sub(data_pack)

[1] "Crossref"
[1] 10.1109/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/
[1] "Crossref"
[1] 10.1111/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/


Warning message in mutate_impl(.data, dots):
“Unequal factor levels: coercing to character”Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "Crossref"
[1] 10.1007/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "Crossref"
[1] 10.1080/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/


...

Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "Crossref"
[1] 10.3892/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

[1] "Crossref"
[1] 10.1207/
298 Levels: 10.1002/ 10.1007/ 10.1016/ 10.1017/ 10.1021/ 10.1023/ ... 10.7906/


Warning message in mutate_impl(.data, dots):
“binding character and factor vector, coercing into character vector”

In [199]:
add_datacenters_labes<-function(data, dc_labels,xr_labels){
    data <- data %>% 
        rowwise() %>% 
        left_join(xr_labels)
    data <- data %>% 
        rowwise() %>% 
        left_join(dc_labels)
    print(nrow(data))
    return(data)
}
proce_data_file<-add_datacenters_labes(data_pack,dd,dd2)

Joining, by = "subj_prefix"
Warning message:
“Column `subj_prefix` joining character vector and factor, coercing into character vector”Joining, by = "obj_prefix"
Warning message:
“Column `obj_prefix` joining character vector and factor, coercing into character vector”

[1] 22000


In [200]:
file<-paste0("../data/",Sys.Date(),"_links_dc_to_xref_with_labels.Rda")
print(file)
save(proce_data_file,file=file)